# Scraping articles by Denisse Dresser

In [71]:
import time
import re

import pandas as pd
import numpy as np

import requests
from urllib.request import urlopen
from bs4 import BeautifulSoup

In [72]:
# Get links of all articles
html = urlopen('https://denisedresser.com/columnas', )
soup = BeautifulSoup(html.read(), 'lxml')
links = []
for column in soup.find_all('div', {'class': 'column'}):
    link = column.a.attrs['href']
    links.append(link)

In [73]:
def dresser_content_scrapper(link):
    
    """Get title, date, and body of an article of Denisse Dresser"""
    
    # Get text of website
    html = requests.get(link, headers=
                 {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:86.0) Gecko/20100101 Firefox/86.0',
                  'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
                  'Accept-Language': 'en-US,en;q=0.5',
                  'Accept-Encoding': 'gzip, deflate, br',
                  'Referer': 'https://denisedresser.com/columnas',
                  'Connection': 'keep-alive',
                  'Upgrade-Insecure-Requests': '1',
                  'Cache-Control': 'max-age=0'}).text
    soup = BeautifulSoup(html, 'lxml')
    
    # Get title
    title = soup.find('h2').get_text()
    
    # Get date
    date = soup.find_all('div', {'class': 'date'})[0].get_text()
    date_clean = date.strip()
    
    # Get body of article
    body = str(soup.find_all('p')) 
    
    # Return
    return title, date_clean, body

In [74]:
# Create variables to store information
data_dresser = {'title': '', 'date': '', 'body': ''}
titles = []
dates = []
bodies = []

In [75]:
# Store information
for link in links:
    title, date, body = dresser_content_scrapper(link)
    time.sleep(1)
    titles.append(title)
    dates.append(date)
    bodies.append(body)

In [76]:
# Format information
data_dresser['title'] = titles
data_dresser['date'] = dates
data_dresser['body'] = bodies
data_dresser

In [80]:
data = pd.DataFrame(data_dresser)
data.head(10)

,title,date,body
0,Los seducidos,"1 de marzo, 2021",[<p>En 2012 el regreso del PRI parecía impensa...
1,El país de AMLO,"22 de febrero, 2021",[<p>La numeralia esencial de un país en riesgo...
2,El show,"15 de febrero, 2021",[<p>Bienvenidos a la carpa de la 4T. Al show m...
3,Mujeres menú,"8 de febrero, 2021","[<p>""Si no tienes un lugar en la mesa, probabl..."
4,Victorias pírricas,"1 de febrero, 2021",[<p>Después de varios días de desaparición y e...
5,Vacunas a voluntad,"25 de enero, 2021","[<p>El jueves 21 de enero hubo 1,803 fallecido..."
6,El buscapleitos,"18 de enero, 2021","[<p>Helo ahí, con el puño en alto, preparándos..."
7,El Arrebatador,"11 de enero, 2021",[<p>Semana tras semana el gobierno de López Ob...
8,Credo por México,"4 de enero, 2021",[<p>Días de desasosiego. Días de incertidumbre...
9,Irresponsables,"28 de diciembre, 2020",[<p>A pesar de la llegada esperanzadora de la ...


In [81]:
data['source'] = 'Personal website'
data['link'] = links
data.insert(0, 'author', 'Denisse Dresser')
data.head()

,author,title,date,body,source,link
0,Denisse Dresser,Los seducidos,"1 de marzo, 2021",[<p>En 2012 el regreso del PRI parecía impensa...,Personal website,https://denisedresser.com/columnas/los-seducidos
1,Denisse Dresser,El país de AMLO,"22 de febrero, 2021",[<p>La numeralia esencial de un país en riesgo...,Personal website,https://denisedresser.com/columnas/el-pais-de-...
2,Denisse Dresser,El show,"15 de febrero, 2021",[<p>Bienvenidos a la carpa de la 4T. Al show m...,Personal website,https://denisedresser.com/columnas/el-show-deb...
3,Denisse Dresser,Mujeres menú,"8 de febrero, 2021","[<p>""Si no tienes un lugar en la mesa, probabl...",Personal website,https://denisedresser.com/columnas/mujeres-menu
4,Denisse Dresser,Victorias pírricas,"1 de febrero, 2021",[<p>Después de varios días de desaparición y e...,Personal website,https://denisedresser.com/columnas/victorias-p...


In [82]:
# Save as csv
data.to_csv('../Data/data_Denisse_Dresser', index=False)